In [24]:
!pip install sentence_transformers evaluate bert_score > log

In [21]:
import numpy as np
import json
import os
import time
import torch
from typing import List

import nltk
from nltk.translate.bleu_score import sentence_bleu
from evaluate import load
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline, AutoConfig, T5Tokenizer, T5ForConditionalGeneration

In [25]:
nltk.download('punkt')
nltk.download('perluniprops')
bertscore = load("bertscore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!


In [16]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
tokenizer = AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_base_commit_generation")
model = AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_base_commit_generation")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)
max_source_length = 512

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/797k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1595: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [9]:
def make_embedding(input_sequences):
  """
  Make embedding from git diff using encoder.
  """

  encoding = tokenizer(
      input_sequences,
      padding="longest",
      max_length=max_source_length,
      truncation=True,
      return_tensors="pt",
  ).to(device)

  input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

  output = model.encoder(input_ids=input_ids, attention_mask=attention_mask)
  pooled_sentence = output.last_hidden_state
  pooled_sentence = torch.mean(pooled_sentence, dim=1)

  return pooled_sentence

In [10]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

def load_data(path):
    with open(path, 'r') as f:
        lines = f.read().split('\n')[0:-1]
    lines = [l.strip() for l in lines]
    return lines


def find_mixed_nn(simi, diffs, test_diff, bleu_thre: int = 5) -> int:
    candidates = simi.argsort()[-bleu_thre:][::-1]
    max_score = 0
    max_idx = 0

    for j in candidates:
      score = sentence_bleu([diffs[j].split()], test_diff.split())
      if score > max_score:
          max_score = score
          max_idx = j
    msg = None
    return max_idx, msg


def find_nn(simi) -> int:
    max_idx = simi.argsort()[-1]
    return max_idx

def nngen(train_diffs: List[str], train_msgs: List[str], test_diffs: List[str], bleu_thre=5) -> List[str]:
    batch_size = 8
    # train_matrix = np.stack([make_embedding(train_batch).cpu().detach().numpy() for train_batch in batch(train_diffs, batch_size) if len(train_batch) == batch_size])
    # test_matrix = np.stack([make_embedding(test_batch).cpu().detach().numpy() for test_batch in batch(test_diffs, batch_size) if len(test_batch) == batch_size])

    train_matrix = np.load('/content/drive/MyDrive/train_mx_22112.npy')
    test_matrix = np.load('/content/drive/MyDrive/test_mx_2520.npy')

    similarities = cosine_similarity(
        np.reshape(test_matrix, (test_matrix.shape[0] * test_matrix.shape[1], test_matrix.shape[2])),
        np.reshape(train_matrix, (train_matrix.shape[0] * train_matrix.shape[1], train_matrix.shape[2])),
    )

    test_msgs = []
    for idx, test_simi in enumerate(similarities):
        max_idx, msg = find_mixed_nn(test_simi, train_diffs, test_diffs[idx], bleu_thre)
        if msg is not None:
          test_msgs.append(msg)
        else:
          test_msgs.append(train_msgs[max_idx])
    return test_msgs


train_limit = 22112
test_limit = 2520
diffs_train = load_data("/content/drive/MyDrive/cleaned.train.diff")[:train_limit]
diffs_test = load_data("/content/drive/MyDrive/cleaned.test.diff")[:test_limit]

target_train = load_data("/content/drive/MyDrive/cleaned.train.msg")[:train_limit]
target_test = load_data("/content/drive/MyDrive/cleaned.test.msg")[:test_limit]

pred_test = nngen(diffs_train, target_train, diffs_test)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [27]:
import string
from nltk import word_tokenize
from nltk.tokenize.nist import NISTTokenizer

nist = NISTTokenizer()

weights = (0.25,0.25,0.25,0.25,)
bleu = []
refs = []
preds = []

for target, pred in zip(target_test, pred_test):
  tokenized = nist.tokenize(target)
  if len(tokenized) >= 4:
    weights = (0.25,0.25,0.25,0.25,)
  elif len(tokenized) == 3:
    weights = (1/3,1/3,1/3,)
  elif len(tokenized) == 2:
    weights = (0.5,0.5,)
  else:
    weights = (1.,)
  score = sentence_bleu(
      [[t.lower() for idx, t in enumerate(nist.tokenize(target))]],
       [t.lower() for idx, t in enumerate(nist.tokenize(pred))],
      weights=weights,
    )
  bleu.append(score)

print("mean BLEU:" , sum(bleu) / len(bleu))


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

mean BLEU: 0.12252056092911812


In [37]:
bert_score = bertscore.compute(predictions=pred_test, references=target_test, lang="en")
sum(bert_score["f1"]) / len(bert_score["f1"])

0.8711475553730178

In [38]:
nn_pred = load_data("/content/drive/MyDrive/nngen.cleaned.test.msg")[:test_limit]

In [39]:
# NNGen

import string
from nltk import word_tokenize
from nltk.tokenize.nist import NISTTokenizer

nist = NISTTokenizer()


weights = (0.25,0.25,0.25,0.25,)
arr = []
b = []
c = 0
for target, pred in zip(target_test, nn_pred):
  tokenized = nist.tokenize(target)
  if len(tokenized) >= 4:
    weights = (0.25,0.25,0.25,0.25,)
  elif len(tokenized) == 3:
    weights = (1/3,1/3,1/3,)
  elif len(tokenized) == 2:
    weights = (0.5,0.5,)
  else:
    weights = (1.,)
  score = sentence_bleu(
      [[t.lower() for idx, t in enumerate(nist.tokenize(target))]],
       [t.lower() for idx, t in enumerate(nist.tokenize(pred))],
      weights=weights,
    )
  arr.append(score)
print("mean:" , sum(arr) / len(arr))

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

mean: 0.12091586609125143


In [40]:
bert_score = bertscore.compute(predictions=nn_pred, references=target_test, lang="en")
sum(bert_score["f1"]) / len(bert_score["f1"])

0.8735183466994573